In [1]:
require 'torch'
require 'nn'
require 'optim'
require 'loadcaffe'
require 'hdf5'

In [2]:
params = {gpu = 0,backend = 'cudnn', 
    caffe_model = '/home/fanfanda/style_transfer/fanfanda_neuralImages/NeuralImageSynthesis/Models/VGG_ILSVRC_19_layers_conv.caffemodel', 
    input_file = '/home/fanfanda/style_transfer/fanfanda_neuralImages/NeuralImageSynthesis/Tmp/input.hdf5', 
    init_file = '/home/fanfanda/style_transfer/fanfanda_neuralImages/NeuralImageSynthesis/Tmp/init.hdf5',
    max_iter = 50, print_iter = 50, save_iter = 0, layer_order = 'relu1_1,relu2_1,relu3_1,relu4_1,relu5_1,relu4_2',
    output_file = '/home/fanfanda/style_transfer/fanfanda_neuralImages/NeuralImageSynthesis/Tmp/output.hdf', 
    mask_file = 'path/to/HDF5file', loss_file = 'path/to/HDF5file'}

In [3]:
paths.dofile('LossLayers.lua')
paths.dofile('Misc.lua')

In [4]:
-- Set gpu mode
if params.gpu >= 0 then
    require 'cutorch'
    require 'cunn'
    cutorch.setDevice(params.gpu + 1)
else
    params.backend = 'nn'
end
if params.backend == 'cudnn' then
    require 'cudnn'
    if params.cudnn_autotune then
        cudnn.benchmark = true
    end
    cudnn.SpatialConvolution.accGradParameters = nn.SpatialConvolutionMM.accGradParameters -- ie: nop
end

In [5]:
-- Load network from caffemodel
local loadcaffe_backend = params.backend
cnn = loadcaffe.load('network', params.caffe_model, params.backend):float()
cnn = set_datatype(cnn, params.gpu)

-- Load optimisation targets 
local f = hdf5.open(params.input_file, 'r')
opt_targets = f:all()
f:close()

Successfully loaded /home/fanfanda/style_transfer/fanfanda_neuralImages/NeuralImageSynthesis/Models/VGG_ILSVRC_19_layers_conv.caffemodel


conv1_1: 64 3 3 3
conv1_2: 64 64 3 3
conv2_1: 128 64 3 3
conv2_2: 128 128 3 3


conv3_1: 256 128 3 3
conv3_2: 256 256 3 3


conv3_3: 256 256 3 3


conv3_4: 256 256 3 3


...

conv4_4: 512 512 3 3


conv5_1: 512 512 3 3


conv5_2: 512 512 3 3


conv5_3: 512 512 3 3


conv5_4: 512 512 3 3


In [6]:
opt_targets

{
  relu2_1 : 
    {
      GramMSE : 
        {
          targets : DoubleTensor - size: 1x128x128
          weights : DoubleTensor - size: 1
        }
    }
  relu3_1 : 
    {
      GramMSE : 
        {
          targets : DoubleTensor - size: 1x256x256
          weights : DoubleTensor - size: 1
        }
    }
  relu1_1 : 
    {
      GramMSE : 
        {
          targets : DoubleTensor - size: 1x64x64
          weights : DoubleTensor - size: 1
        }
    }
  relu5_1 : 
    {
      GramMSE : 
        {
          targets : DoubleTensor - size: 1x512x512
          weights : DoubleTensor - size: 1
        }
    }
  relu4_1 : 
    {
      GramMSE : 
        {
          targets : DoubleTensor - size: 1x512x512
          weights : DoubleTensor - size: 1


        }
    }
  relu4_2 : 
    {
      MSE : 
        {
          targets : DoubleTensor - size: 1x512x56x74
          weights : LongTensor - size: 1
        }
    }
}


In [6]:
-- Set up new network with appropriate loss layers
net = nn.Sequential()
loss_modules = {}
next_layer_ndx = 1
-- Loss layers acting directly on the image
if opt_targets['data'] then
    loss_modules['data'] = {}
    for loss_layer, args in pairs(opt_targets['data']) do
        local loss_module = get_loss_module(loss_layer, args)
        loss_module = set_datatype(loss_module, params.gpu)
        net:add(loss_module)
        loss_modules['data'][loss_layer] = loss_module
    end
    next_layer_ndx = next_layer_ndx + 1
end

In [7]:
-- Loss layers acting on CNN features
for i = 1, #cnn do
    if next_layer_ndx <= length(opt_targets) then
        local layer = cnn:get(i)
        local layer_name = layer.name
        local layer_type = torch.type(layer)
        local is_convolution = (layer_type == 'cudnn.SpatialConvolution' or layer_type == 'nn.SpatialConvolution')
        if is_convolution and params.reflectance then
            local padW, padH = layer.padW, layer.padH
            local pad_layer = nn.SpatialReflectionPadding(padW, padW, padH, padH)
            pad_layer = set_datatype(pad_layer, params.gpu)
            net:add(pad_layer)
            layer.padW = 0
            layer.padH = 0
        end
        net:add(layer)
        if opt_targets[layer_name] then
            loss_modules[layer_name] = {}
            for loss_layer, args in pairs(opt_targets[layer_name]) do
                if loss_layer == 'GramMSEDilation' then
                    args['conv_layer'] = net.modules[#net.modules-1]
                    local dilation_losses = get_loss_module(loss_layer, args)
                    for i, dl in ipairs(dilation_losses) do 
                        dl = set_datatype(dl, params.gpu)
                        table.insert(net.modules, #net.modules-1, dl)
                    end
                    loss_modules[layer_name][loss_layer] = dilation_losses
                else
                    local loss_module = get_loss_module(loss_layer, args)
                    loss_module = set_datatype(loss_module, params.gpu)
                    net:add(loss_module)
                    loss_modules[layer_name][loss_layer] = loss_module
                end
            end
            next_layer_ndx = next_layer_ndx + 1
        end
    end
end

In [8]:
-- Get flat list of loss modules to call in feval
loss_modules_flat = {}
for layer_name, layer_table in pairs(loss_modules) do
    for loss_layer, loss_module in pairs(layer_table) do
        if loss_layer == 'GramMSEDilation' then
            for _, dilation_module in pairs(loss_module) do
                loss_modules_flat[#loss_modules_flat + 1] = dilation_module
            end
        else
            loss_modules_flat[#loss_modules_flat + 1] = loss_module
        end
    end
end

In [9]:
-- We don't need the base CNN anymore, so clean it up to save memory.
cnn = nil
for i=1, #net.modules do
    local module = net.modules[i]
    if torch.type(module) == 'nn.SpatialConvolutionMM' then
        module.gradWeight = nil
        module.gradBias = nil
    end
end
collectgarbage()

In [10]:
-- Load initialisation 
local f = hdf5.open(params.init_file, 'r')
img = f:all()['init']
f:close()
img = set_datatype(img, params.gpu)

In [11]:
-- Load mask if specified
mask = nil
if params.mask_file ~= 'path/to/HDF5file' then
    local f = hdf5.open(params.mask_file, 'r')
    mask = f:all()['mask']
    f:close()
    mask = set_datatype(mask, params.gpu)
end

In [12]:
-- Run it through the network once to get the proper size for the gradient
-- All the gradients will come from the extra loss modules, so we just pass
-- zeros into the top of the net on the backward pass.
y = net:forward(img)
dy = img.new(#y):zero()

-- Declare optimisation options
optim_state = {
  maxIter = params.max_iter,
  verbose = true,
  tolX = 0,
  tolFun = 0,
}

-- Get layer_order for use in maybe_print
layer_order = params.layer_order:split(",")

In [13]:
-- Function to evaluate loss and gradient. We run the net forward and
-- backward to get the gradient, and sum up losses from the loss modules.
-- optim.lbfgs internally handles iteration and calls this fucntion many
-- times, so we manually count the number of iterations to handle printing
-- and saving intermediate results.
num_calls = 0
function feval(x)
    num_calls = num_calls + 1
    net:forward(x)
    local grad = net:updateGradInput(x, dy)
    local loss = 0
    for _, mod in ipairs(loss_modules_flat) do
        loss = loss + mod.loss
    end
    maybe_print(num_calls, params.print_iter, params.max_iter, layer_order, loss_modules, loss)
    maybe_save(num_calls, params.save_iter, params.max_iter, params.output_file, img)

    if mask then
        grad[mask:repeatTensor(1,1,1):expandAs(grad)] = 0
    end

    collectgarbage()
    -- optim.lbfgs expects a vector for gradients
    return loss, grad:view(grad:nElement())
end

In [14]:
 -- Run optimization.
print('Running optimization with L-BFGS')
local x, losses = optim.lbfgs(feval, img, optim_state)

-- Also save result if optimisation stops before max iter is reached
if num_calls < params.max_iter then
    maybe_save(params.max_iter, params.save_iter, params.max_iter, params.output_file, img)
end

-- Optionally save the loss as tracked over the optimisation
if params.loss_file ~= 'path/to/HDF5file' then
    local f = hdf5.open(params.loss_file, 'w')
    f:write('losses', torch.Tensor(losses):double())
    f:close()
end

Running optimization with L-BFGS	


<optim.lbfgs> 	creating recyclable direction/step/history buffers	


Iteration 50 / 50	
relu1_1	
GramMSE loss: 43328.506470	
relu2_1	
GramMSE loss: 313642.517090	
relu3_1	
GramMSE loss: 90680.923462	
relu4_1	
GramMSE loss: 509457.916260	
relu5_1	
GramMSE loss: 6963.839531	
relu4_2	
MSE loss: 369163.031250	
Total loss: 1333236.734062	


<optim.lbfgs> 	reached max number of iterations	


In [16]:
?optim.lbfgs

+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++	
lbfgs(opfunc, x[, config][, state])


An implementation of L-BFGS that relies on a user-provided line search 
function ( state.lineSearch ).
If this function is not provided, then a simple learning rate is used 
to produce fixed size steps.
Fixed size steps are much less costly than line searches, and can be useful 
for stochastic problems.

The learning rate is used even when a line search is provided.
This is also useful for large-scale stochastic problems, where opfunc 
is a noisy approximation of f(x) .
In that case, the learning rate allows a reduction of confidence in the 
step size.

Arguments:

>  opfunc : a function that takes a single input X , the point of 
evaluation, and returns f(X) and df/dX >  x : the initial point
>  config : a table with configuration parameters for the optimizer
>  config.maxIter : Maximum number of iterations allowed
>  config.maxEval : Maximum number of function evaluations
>  c